# 第3章: 正規表現
Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

* 1行に1記事の情報がJSON形式で格納される
* 各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
* ファイル全体はgzipで圧縮される
以下の処理を行うプログラムを作成せよ．

## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [1]:
import gzip
import json

with gzip.open('./data/jawiki-country.json.gz', mode='rt') as f:
    jsons = []
    # 元データがJSON Lines≠JSONのため、１行ずつ読み込み
    lines = f.readlines() 
    for line in lines:
        jsons.append(json.loads(line))
    # イギリスを抽出
    eng = list(filter(lambda e: e['title'] == 'イギリス', jsons))
    eng_data = eng[0]

## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [2]:
import regex as re

# 本文を抽出
texts = eng_data['text'].split('\n')

# カテゴリを含む行を抽出
cat_rows = list(filter(lambda e: re.search('\[\[カテゴリ:|\[\[Category:', e), texts))
print('\n'.join(cat_rows))

[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]


## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [3]:
# カテゴリを含む行からカテゴリ名のみを抽出
cat_rows = list(map(lambda e: re.search('(?<=(\[\[カテゴリ:|\[\[Category:)).+?(?=(\||\]))', e).group(), cat_rows))
print('\n'.join(cat_rows))

イギリス
イギリス連邦加盟国
英連邦王国
G8加盟国
欧州連合加盟国
海洋国家
現存する君主国
島国
1801年に成立した国家・領域


## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [4]:
# セクションを含む行を抽出
sec_rows = list(filter(lambda e: re.search('==.+==', e), texts))

# =の数からレベルを算出
sec_rows_num = list(map(lambda e: e + ':' + str(int(e.count('=') / 2 - 1)), sec_rows))

# =と空白を削除
sections = list(map(lambda e: e.replace('=', '').replace(' ', ''), sec_rows_num))
print('\n'.join(sections))

国名:1
歴史:1
地理:1
主要都市:2
気候:2
政治:1
元首:2
法:2
内政:2
地方行政区分:2
外交・軍事:2
経済:1
鉱業:2
農業:2
貿易:2
不動産:2
エネルギー政策:2
通貨:2
企業:2
通信:3
交通:1
道路:2
鉄道:2
海運:2
航空:2
科学技術:1
国民:1
言語:2
宗教:2
婚姻:2
移住:2
教育:2
医療:2
文化:1
食文化:2
文学:2
哲学:2
音楽:2
ポピュラー音楽:3
映画:2
コメディ:2
国花:2
世界遺産:2
祝祭日:2
スポーツ:2
サッカー:3
クリケット:3
競馬:3
モータースポーツ:3
野球:3
カーリング:3
自転車競技:3
脚注:1
関連項目:1
外部リンク:1


## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [5]:
# ファイルを含む行を抽出
file_rows = list(filter(lambda e: re.search('\[\[ファイル:|\[\[File:|\[\[file:', e), texts))

# ファイルを含む行からファイル名のみを抽出
file_rows = list(map(lambda e: re.search('(?<=(\[\[ファイル:|\[\[File:|\[\[file:)).+?(?=(\||\]))', e).group(), file_rows))
print('\n'.join(file_rows))

Royal Coat of Arms of the United Kingdom.svg
United States Navy Band - God Save the Queen.ogg
Descriptio Prime Tabulae Europae.jpg
Lenepveu, Jeanne d'Arc au siège d'Orléans.jpg
London.bankofengland.arp.jpg
Battle of Waterloo 1815.PNG
Uk topo en.jpg
BenNevis2005.jpg
Population density UK 2011 census.png
2019 Greenwich Peninsula & Canary Wharf.jpg
Leeds CBD at night.jpg
Palace of Westminster, London - Feb 2007.jpg
Scotland Parliament Holyrood.jpg
Donald Trump and Theresa May (33998675310) (cropped).jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
City of London skyline from London City Hall - Oct 2008.jpg
Oil platform in the North SeaPros.jpg
Eurostar at St Pancras Jan 2008.jpg
Heathrow Terminal 5C Iwelumo-1.jpg
UKpop.svg
Anglospeak.svg
Royal Aberdeen Children's Hospital.jpg
CHANDOS3.jpg
The Fabs.JPG
Wembley Stadium, illuminated.jpg


## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [6]:
import pprint

# 本文を抽出
text = eng_data['text']

# 基礎情報を抽出
basic_text = re.search('{{基礎情報[\s\S]+?}}\n\n', text).group().replace('\n*', '*')

# 改行区切りでリスト化し、不要部分を削除
basic_ary = basic_text.split('\n')
del basic_ary[0]
del basic_ary[-3:]

# 辞書型に変更
basic_dict = {}
for basic in basic_ary:
    key, *values = basic.split('=')
    key = key.replace(' ', '').replace('|', '')
    basic_dict[key] = ''.join(values).strip()
pprint.pprint(basic_dict)

{'GDP/人': '36,727<ref name"imf-statistics-gdp" />',
 'GDP値': '2兆3162億<ref name"imf-statistics-gdp" />',
 'GDP値MER': '2兆4337億<ref name"imf-statistics-gdp" />',
 'GDP値元': '1兆5478億<ref '
          'name"imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x70&pr.y13&sy2010&ey2012&scsm1&ssd1&sortcountry&ds.&br1&c112&sNGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp0&aIMF>Data '
          'and Statistics>World Economic Outlook Databases>By Countrise>United '
          'Kingdom]</ref>',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '6',
 'ISO3166-1': 'GB / GBR',
 'ccTLD': '[[.uk]] / [[.gb]]<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': '6643万5600<ref>{{Cite '
        'web|urlhttps://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates|titlePopulation '
        'estimates - Office for National '
        'Statistics|accessdate2019-06-26|date2019-06-26}}</ref>',
 '人口大きさ': '1 E7',
 '人口密度値': '271',
 '人口統計年

## 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: マークアップ早見表）．

In [7]:
# 基礎情報を抽出
basic_text = re.search('{{基礎情報[\s\S]+?}}\n\n', text).group().replace('\n*', '*')

# 改行区切りでリスト化し、不要部分を削除
basic_ary = basic_text.split('\n')
del basic_ary[0]
del basic_ary[-3:]

# 辞書型に変更
basic_dict = {}
for basic in basic_ary:
    # キーと値に分割
    key, *values = basic.split('=')
    # キーを整形
    key = key.replace(' ', '').replace('|', '')
    # 値がリストになっているため結合
    value = ''.join(values).strip()
    # 強調マークアップの除去
    value = value.replace("'''''", '').replace("'''", '').replace("''", '')
    basic_dict[key] = value
pprint.pprint(basic_dict)

{'GDP/人': '36,727<ref name"imf-statistics-gdp" />',
 'GDP値': '2兆3162億<ref name"imf-statistics-gdp" />',
 'GDP値MER': '2兆4337億<ref name"imf-statistics-gdp" />',
 'GDP値元': '1兆5478億<ref '
          'name"imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x70&pr.y13&sy2010&ey2012&scsm1&ssd1&sortcountry&ds.&br1&c112&sNGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp0&aIMF>Data '
          'and Statistics>World Economic Outlook Databases>By Countrise>United '
          'Kingdom]</ref>',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '6',
 'ISO3166-1': 'GB / GBR',
 'ccTLD': '[[.uk]] / [[.gb]]<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': '6643万5600<ref>{{Cite '
        'web|urlhttps://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates|titlePopulation '
        'estimates - Office for National '
        'Statistics|accessdate2019-06-26|date2019-06-26}}</ref>',
 '人口大きさ': '1 E7',
 '人口密度値': '271',
 '人口統計年

## 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．

In [8]:
# 基礎情報を抽出
basic_text = re.search('{{基礎情報[\s\S]+?}}\n\n', text).group().replace('\n*', '*')

# 改行区切りでリスト化し、不要部分を削除
basic_ary = basic_text.split('\n')
del basic_ary[0]
del basic_ary[-3:]

# 辞書型に変更
basic_dict = {}
for basic in basic_ary:
    # キーと値に分割
    key, *values = basic.split('=')
    # キーを整形
    key = key.replace(' ', '').replace('|', '')
    # 値がリストになっているため結合
    value = ''.join(values).strip()
    # 強調マークアップの除去
    value = value.replace("'''''", '').replace("'''", '').replace("'", '')
    # 内部リンク文字列を取得
    targets = re.findall('((?<=({{)).+?(?=(}})))', value)
    # 内部リンク文字列の整形
    if targets:
        for target in targets:
            value = re.sub('{{.+?}}', target[0].split('|')[-1], value, count=1)
    # 内部リンク文字列を取得
    targets = re.findall('((?<=(\[\[)).+?(?=(\]\])))', value)
    # 内部リンク文字列の整形
    if targets:
        for target in targets:
            value = re.sub('\[\[.+?\]\]', target[0].split('|')[-1], value, count=1)
    basic_dict[key] = value
pprint.pprint(basic_dict)

{'GDP/人': '36,727<ref name"imf-statistics-gdp" />',
 'GDP値': '2兆3162億<ref name"imf-statistics-gdp" />',
 'GDP値MER': '2兆4337億<ref name"imf-statistics-gdp" />',
 'GDP値元': '1兆5478億<ref '
          'name"imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x70&pr.y13&sy2010&ey2012&scsm1&ssd1&sortcountry&ds.&br1&c112&sNGDP%2CNGDPD%2CPPPGDP%2CPPPPC&grp0&aIMF>Data '
          'and Statistics>World Economic Outlook Databases>By Countrise>United '
          'Kingdom]</ref>',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '6',
 'ISO3166-1': 'GB / GBR',
 'ccTLD': '.uk / .gb<ref>使用は.ukに比べ圧倒的少数。</ref>',
 '人口値': '6643万5600<ref>date2019-06-26</ref>',
 '人口大きさ': '1 E7',
 '人口密度値': '271',
 '人口統計年': '2018',
 '人口順位': '22',
 '他元首等氏名1': 'ノーマン・ファウラー',
 '他元首等氏名2': 'Lindsay Hoyle',
 '他元首等氏名3': 'ブレンダ・ヘイル',
 '他元首等肩書1': '貴族院議長',
 '他元首等肩書2': '庶民院議長',
 '他元首等肩書3': '最高裁判所長官',
 '位置画像': 'United Kingdom (+overseas territories) in the World 

## 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [9]:
# 本文を抽出
text = eng_data['text']

# 基礎情報を抽出
basic_text = re.search('{{基礎情報[\s\S]+?}}\n\n', text).group().replace('\n*', '*')

# 改行区切りでリスト化し、不要部分を削除
basic_ary = basic_text.split('\n')
del basic_ary[0]
del basic_ary[-3:]

# 字書型に変更
basic_dict = {}
for basic in basic_ary:
    # キーと値に分割
    key, *values = basic.split('=')
    # キーを整形
    key = key.replace(' ', '').replace('|', '')
    # 値がリストになっているため結合
    value = ''.join(values).strip()
    # 強調マークアップの除去
    value = value.replace("'''''", '').replace("'''", '').replace("'", '')
    # 内部リンク文字列を取得
    targets = re.findall('((?<=({{)).+?(?=(}})))', value)
    # 内部リンク文字列の整形
    if targets:
        for target in targets:
            value = re.sub('{{.+?}}', target[0].split('|')[-1], value, count=1)
    # 内部リンク文字列を取得
    targets = re.findall('((?<=(\[\[)).+?(?=(\]\])))', value)
    # 内部リンク文字列の整形
    if targets:
        for target in targets:
            value = re.sub('\[\[.+?\]\]', target[0].split('|')[-1], value, count=1)
    # タグ除去
    value = value.replace('<br />', '')
    value = re.sub('<ref.+?</ref>', '', value)
    value = re.sub('<ref.+?/>', '', value)
    basic_dict[key] = value
pprint.pprint(basic_dict)

{'GDP/人': '36,727',
 'GDP値': '2兆3162億',
 'GDP値MER': '2兆4337億',
 'GDP値元': '1兆5478億',
 'GDP統計年': '2012',
 'GDP統計年MER': '2012',
 'GDP統計年元': '2012',
 'GDP順位': '6',
 'GDP順位MER': '6',
 'ISO3166-1': 'GB / GBR',
 'ccTLD': '.uk / .gb',
 '人口値': '6643万5600',
 '人口大きさ': '1 E7',
 '人口密度値': '271',
 '人口統計年': '2018',
 '人口順位': '22',
 '他元首等氏名1': 'ノーマン・ファウラー',
 '他元首等氏名2': 'Lindsay Hoyle',
 '他元首等氏名3': 'ブレンダ・ヘイル',
 '他元首等肩書1': '貴族院議長',
 '他元首等肩書2': '庶民院議長',
 '他元首等肩書3': '最高裁判所長官',
 '位置画像': 'United Kingdom (+overseas territories) in the World (+Antarctica '
         'claims).svg',
 '元首等氏名': 'エリザベス2世',
 '元首等肩書': '女王',
 '公式国名': 'United Kingdom of Great Britain and Northern Ireland',
 '公用語': '英語',
 '国旗画像': 'Flag of the United Kingdom.svg',
 '国歌': 'God Save the Queenen icon神よ女王を護り賜えファイル:United States Navy Band - God '
       'Save the Queen.ogg',
 '国章リンク': '（国章）',
 '国章画像': 'イギリスの国章',
 '国際電話番号': '44',
 '地図画像': 'Europe-UK.svg',
 '夏時間': '+1',
 '建国形態': '建国',
 '日本語国名': 'グレートブリテン及び北アイルランド連合王国',
 '時間帯': '±0',
 '最大都市': 'ロンド

## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [10]:
import requests

# 基礎情報を抽出
basic_text = re.search('{{基礎情報[\s\S]+?}}\n\n', text).group().replace('\n*', '*')

# 改行区切りでリスト化し、不要部分を削除
basic_ary = basic_text.split('\n')
del basic_ary[0]
del basic_ary[-3:]

# 字書型に変更
basic_dict = {}
for basic in basic_ary:
    # キーと値に分割
    key, *values = basic.split('=')
    # キーを整形
    key = key.replace(' ', '').replace('|', '')
    # 値がリストになっているため結合
    value = ''.join(values).strip()
    # 強調マークアップの除去
    value = value.replace("'''''", '').replace("'''", '').replace("'", '')
    # 内部リンク文字列を取得
    targets = re.findall('((?<=({{)).+?(?=(}})))', value)
    # 内部リンク文字列の整形
    if targets:
        for target in targets:
            value = re.sub('{{.+?}}', target[0].split('|')[-1], value, count=1)
    # 内部リンク文字列を取得
    targets = re.findall('((?<=(\[\[)).+?(?=(\]\])))', value)
    # 内部リンク文字列の整形
    if targets:
        for target in targets:
            value = re.sub('\[\[.+?\]\]', target[0].split('|')[-1], value, count=1)
    # タグ除去
    value = value.replace('<br />', '')
    value = re.sub('<ref.+?</ref>', '', value)
    value = re.sub('<ref.+?/>', '', value)
    basic_dict[key] = value

# API呼び出し
session = requests.Session()
params = {
    'action': 'query',
    'format': 'json',
    'prop': 'imageinfo',
    'titles': 'File:' + basic_dict['国旗画像'],
    'iiprop': 'url'
}

result = session.get('https://ja.wikipedia.org/w/api.php', params=params)
res_json = result.json()
print(res_json['query']['pages']['-1']['imageinfo'][0]['url'])

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
